In [1]:
import pandas as pd
import numpy as np
import random as rd
from typing import Literal, Tuple
from matplotlib import pyplot as plt
from scipy.optimize import linprog
import time

In [2]:
class Game(pd.DataFrame):
    def __init__(self, payoff_df:pd.DataFrame, deep = True) -> None:
        super().__init__()
        for column in payoff_df.columns:
            this_column:pd.Series = payoff_df[column]
            self[column] = this_column.copy(deep = deep)
            
    def copy(self, deep = True):
        return self.__init__(payoff_df = self, deep = deep)    
    
    def outcome(self, strategies:tuple) -> Tuple[float, float]:
        if len(strategies) < 2:
            raise ValueError("Need at least 2 strategies to fix an outcome. What is given has a length of {}".format(
                len(strategies)))
        s1, s2 = strategies[:2]
        if s1 not in self.index:
            raise ValueError("Can't find strategy '{}' for player 1, available strategies are {}".format(s1, self.index))
        if s2 not in self.columns:
            raise ValueError("Can't find strategy '{}' for player 2, available strategies are {}".format(s2, self.columns))
        return self.loc[s1, s2]
    
    def payoff_matrix(self, player:Literal["row", "column"] = "column") -> np.matrix:
        player = player.lower()
        if player not in ["row", "column"]:
            raise ValueError("Player need to be one of ['row', 'column'], what is given is {}".format(player))
        player_index = {"row":0, "column":1}[player]
        result = []
        for s1 in self.index:
            this_row = []
            for s2 in self.columns:
                this_row += [self.loc[s1, s2][player_index]]
            result += [this_row]
        return np.matrix(result)
    
    def strategies(self, player:Literal["row", "column"] = "column") -> pd.Index:
        player = player.lower()
        if player not in ["row", "column"]:
            raise ValueError("Player need to be one of ['row', 'column'], what is given is {}".format(player))
        player_index = {"row":0, "column":1}[player]
        return [self.index, self.columns][player_index]
    
    def from_matrices(player_1_payoff_matrix:np.matrix,
                    player_2_payoff_matrix:np.matrix, 
                    transposed = False,
                    player_1_strategies:list[str] = list("UD"), 
                    player_2_strategies:list[str] = list("LR")):
        if not transposed: player_2_payoff_matrix = player_2_payoff_matrix.T
        if not player_1_payoff_matrix.shape == player_2_payoff_matrix.shape: raise ValueError
        if not len(player_1_strategies) == player_1_payoff_matrix.shape[0]: raise ValueError
        if not len(player_2_strategies) == player_2_payoff_matrix.shape[0]: raise ValueError
        payoff_dict = {}
        for j, strategy_2 in enumerate(player_2_strategies):
            payoff_dict[strategy_2] = [(player_1_payoff_matrix[i,j], player_2_payoff_matrix[i,j])
                                        for i, strategy_1 in enumerate(player_1_strategies)]
        return Game(payoff_df = pd.DataFrame(payoff_dict, index = player_1_strategies))

In [3]:
def maximin(payoff_matrix:np.matrix, player:Literal["row", "column"] = "column") -> tuple[np.float64, np.matrix]:
    worst:float = payoff_matrix.min()
    A:np.matrix = payoff_matrix + abs(worst) if worst < 0 else payoff_matrix
    if player.lower() == "row": A = A.T
    b = np.ones(A.shape[0])
    c = np.ones(A.shape[1])
    bounds = [(0, None)] * A.shape[1]
    result = linprog(c = c, A_ub = -A, b_ub = -b, bounds = bounds)
    if not result.success: print("Note that the linear programming was not successful...")
    d = result.fun
    maximin_value = d**-1 + worst if worst < 0 else d**-1
    maximin_strategy = np.matrix(result.x).T / d
    return maximin_value, maximin_strategy

In [4]:
def respond_to_mixed_strategy(payoff_matrix:np.matrix, 
                              threshold:float, 
                              opponents_strategy:np.matrix, 
                              player:Literal["row", "column"] = "column", ) -> tuple[np.float64, np.matrix]:
    A = payoff_matrix
    if player.lower() == "row": A = A.T
    v, p = maximin(payoff_matrix = payoff_matrix, player = player)
    if threshold > v: raise ValueError("The threshold is greater than the maximin value...")
    del p
    A:np.matrix = A - threshold 
    # ^ This is A prime now, we only need the 'new payoff vector' to be above 0 to be above the threshold.
    q = opponents_strategy
    if not q.shape == (A.shape[0], 1): raise ValueError(
        "The dimension of the opponent's strategy is not correct: got {}, expecting ({}, 1)".format(
        q.shape, A.shape[0]))
    qTA = (q.T*A).tolist()[0]
    b = np.zeros(A.shape[0])
    c = -np.array(qTA)
    bounds = [(0, None)] * A.shape[1]
    result = linprog(c = c, 
                     A_ub = -A, b_ub = -b, 
                     A_eq = np.ones(shape = (1, A.shape[1])), 
                     b_eq = np.ones(shape = 1), 
                     bounds = bounds)
    if not result.success: print("Note that the linear programming was not successful...")
    maximizing_strategy = np.matrix(result.x).T
    # ^ This is already bounded to the hyperplane of probability vectors
    # maximizing_strategy:np.matrix =  maximizing_vector / maximizing_vector.sum()
    payoff = (qTA*maximizing_strategy)[0,0] + threshold
    return payoff, maximizing_strategy

In [5]:
class LinearProgrammingError(Exception):
    def __init__(self, *args: object) -> None:
        super().__init__(*args)

    def __str__(self) -> str:
        return super().__str__()
    
def nonzero_at(support:np.matrix) -> list:
    return [i for i, row in enumerate(support) if row != [0]]
    
def sub(payoff_matrix:np.matrix, support:np.matrix, 
        player:Literal["row", "column"] = "column") -> np.matrix:
    A, s = payoff_matrix, support
    player = player.lower()
    if player == "row": A = A.T
    if not A.shape[1] == s.shape[0]: raise ValueError("The dimensions are A:{}, s:{}".format(A.shape, s.shape))
    if not s.shape[1] == 1: raise ValueError("The s should be a column vector: dim(s):{}".format(s.shape))
    active_columns = nonzero_at(support = support)
    if player == "row": return A[:, active_columns].T
    return A[:, active_columns]
    
def possible_supports(game:Game, player:Literal["row", "column"] = "column") -> np.matrix:
    rows = []
    S = game.strategies(player = player)
    dim_S = S.shape[0]
    for i in range(1, 2**dim_S):
        this_support_as_str = bin(i)[2:]
        while len(this_support_as_str) < dim_S:
            this_support_as_str = '0' + this_support_as_str
        this_support_as_list = [int(j) for j in this_support_as_str]
        rows.append(this_support_as_list)
    result:np.matrix = np.matrix(rows)
    result_as_list = []
    for row_i in result:
        row_i:np.matrix
        result_as_list.append(row_i.T)
    return result_as_list

def max_indifferent(opponents_payoff_matrix:np.matrix, 
                    opponents_support:np.matrix, 
                    opponents_type:Literal["row", "column"],
                    players_support:np.matrix,
                    ) -> np.matrix:
    A = opponents_payoff_matrix
    opponents_type = opponents_type.lower()
    if opponents_type == "column": A = A.T
    s_opponent = opponents_support
    A_reduced = sub(payoff_matrix = A, 
                    support = opponents_support, 
                    player = "row") # Fix row because we have already transposed
    c = s_opponent.T * A
    A_ub, b_ub = A, np.ones(shape = (A.shape[0], 1))
    A_eq, b_eq = A_reduced, np.ones(shape = (A_reduced.shape[0], 1))
    bounds = [(0, None) if p_i > 0 else (0, 0) for p_i in players_support.T.tolist()[0]]
    result = linprog(c = c, 
                     A_ub = A_ub, b_ub = b_ub, 
                     A_eq = A_eq, b_eq = b_eq, 
                     bounds = bounds)
    if not result.success: raise LinearProgrammingError(
            "The linear programming was not successful...")
    mixed_strategy = np.matrix(result.x).T
    n1_norm = mixed_strategy.sum()
    if not n1_norm > 0: raise ValueError(
            "The result of linear programming returned a non positive vector...")
    mixed_strategy = mixed_strategy / n1_norm
    return mixed_strategy

def support_enumeration(game:Game):
    "Returns the mixed nash equilibria of this game"
    A = game.payoff_matrix(player = "row")
    B = game.payoff_matrix(player = "column")
    A_shift_scalar = abs(A.min())*2
    B_shift_scalar = abs(B.min())*2
    A += A_shift_scalar
    B += B_shift_scalar
    possible_supports_row = possible_supports(game = game, player = "row")
    possible_supports_column = possible_supports(game = game, player = "column")
    good_pairs = []
    for s1 in possible_supports_row[:]:
        for s2 in possible_supports_column[:]:
            s1:np.matrix
            s2:np.matrix
            try:
                p = max_indifferent(opponents_payoff_matrix = B,
                                    opponents_support = s2,
                                    opponents_type = "column",
                                    players_support = s1)
                q = max_indifferent(opponents_payoff_matrix = A,
                                    opponents_support = s1,
                                    opponents_type = "row",
                                    players_support = s2)
            except LinearProgrammingError: continue
            good_pairs.append((p.T, q.T))
    return good_pairs

In [6]:
class Bird:
    strategy_dict = {"hawk":'H', "dove":'D'}
    
    def __init__(self, genotype = rd.choice(["Hawk", "Dove"]))-> None:
        self.genotype:str = genotype
        self.bird_id:str = str(rd.random())[2:]
        self.strategy = Bird.strategy_dict[self.genotype.lower()]
        self.payoff_obtained = 0.0

    def __str__(self) -> str:
        return "{}: {}".format(self.genotype, self.bird_id)
    
    def __repr__(self) -> str:
        return str(self)
    
    def __gt__(self, other) -> bool:
        if isinstance(other, type(self)): return self.bird_id > other.bird_id
        raise TypeError
    
    def __lt__(self, other) -> bool:
        if isinstance(other, type(self)): return self.bird_id < other.bird_id
        raise TypeError
    
    def __eq__(self, other) -> bool:
        if isinstance(other, type(self)): return self.bird_id == other.bird_id
        raise TypeError
    
    def __ge__(self, other) -> bool:
        return self > other or self == other
    
    def __le__(self, other) -> bool:
        return self < other or self == other
    
    def __hash__(self) -> int:
        return hash(str(self))
    
    def child(self):
        return Bird(genotype = self.genotype)

In [7]:
def is_hawk(bird:Bird) -> bool:
    return bird.genotype.lower() == "hawk"

def is_dove(bird:Bird) -> bool:
    return bird.genotype.lower() == "dove"

In [8]:
def encounter(bird_A:Bird, bird_B:Bird, coefficient = 1) -> None:
    for bird_i in [bird_A, bird_B]:
        if not (is_hawk(bird_i) or is_dove(bird_i)): raise TypeError
    if bird_A == bird_B: raise ValueError

    if is_hawk(bird_A) and is_hawk(bird_B):
        bird_A.payoff_obtained -= 25 * coefficient
        bird_B.payoff_obtained -= 25 * coefficient

    elif is_hawk(bird_A) and is_dove(bird_B):
        bird_B.payoff_obtained += 5 * coefficient
        bird_A.payoff_obtained += 45 * coefficient

    elif is_dove(bird_A) and is_hawk(bird_B):
        bird_A.payoff_obtained += 5 * coefficient
        bird_B.payoff_obtained += 45 * coefficient

    elif is_dove(bird_A) and is_dove(bird_B):
        bird_A.payoff_obtained += 15 * coefficient
        bird_B.payoff_obtained += 15 * coefficient

In [9]:
class Population:
    def __init__(self, initial_hawk_frequency = 0.5, size = 64) -> None:
        self.size = size
        self.population = [Bird(genotype = "Hawk") 
                        if rd.random() < initial_hawk_frequency else Bird(genotype = "Dove")
                        for i in range(size)]
        self.evolution_time = 0
        
    def __str__(self) -> str:
        return "Population of size {}: {} ... {}".format(
            self.size, self.population[:2], self.population[-2:])
        
    def __repr__(self) -> str:
        return str(self)
    
    def frequency(self, genotype = "Hawk") -> float:
        cnt = 0
        size = 0
        for player_i in self.population: 
            if player_i.genotype == genotype: cnt += 1
            size += 1
        if size == 0: return np.nan
        return cnt / size

    def shuffle(self) -> None:
        rd.shuffle(self.population)

    def truncate(self, threshold = -25.0, inclusive = False) -> None:
        if inclusive: self.population = [player_i 
            for player_i in self.population if player_i.payoff_obtained > threshold]; return
        self.population = [player_i 
            for player_i in self.population if player_i.payoff_obtained >= threshold]
        
    def reproduce(self) -> None:
        current_population_size = len(self.population)
        if current_population_size <= 0: raise ValueError
        newborn_size = self.size - current_population_size
        if newborn_size < 0: raise ValueError
        current_population = self.population.copy()
        for i in range(newborn_size): self.population += [rd.choice(current_population).child()]

    def reset_payoffs(self, default_value = 0.0) -> None:
        for player_i in self.population: player_i.payoff_obtained = default_value

    def random_encounter(self) -> None:
        if self.size == 1: return
        self.shuffle()
        for i in range(0, len(self.population), 2): 
            player_A, player_B = self.population[i], self.population[i+1]
            encounter(player_A, player_B)

    def round_robin_tournament(self, average_payoff = True) -> None:
        opponent_size = len(self.population) - 1
        coefficient = 1 / opponent_size if average_payoff else 1.0
        for i, player_i in enumerate(self.population[:-1]):
            for player_j in self.population[i+1:]:
                encounter(player_i, player_j, coefficient = coefficient)

    def evolve(self, threshold = 0.0, loop = 8, 
               method:Literal["random", "round_robin"] = "random", 
               reset_before_each_loop = True, 
               initial_payoff = 0.0, 
               inclusive_culls = False) -> bool:
        "The boolean returned indicates if the population still has some survivors"
        if method not in ["random", "round_robin"]: raise ValueError
        for t in range(1, loop + 1):
            if reset_before_each_loop: self.reset_payoffs(default_value = initial_payoff)
            self.shuffle()
            if method == "random": self.random_encounter()
            elif method == "round_robin": self.round_robin_tournament(average_payoff = True)
            if reset_before_each_loop:
                self.truncate(threshold = threshold, inclusive = inclusive_culls)
            else: 
                self.truncate(threshold = threshold*t, inclusive = inclusive_culls) # Returns wrong result 
            if len(self.population) == 0: return False
            self.evolution_time += 1
            self.reproduce()
        return True

In [10]:
HD_payoff_matrix = np.matrix([[-25, 45],
                              [  5, 15]])
A = HD_payoff_matrix

def safety_zone(threshold = 15) -> list[np.matrix, np.matrix]:
    '''Because the game we are analyzing is 2x2, the convex hull is a line, 
        so we only need to compute the line segement where this line falls in the first orthant(quadrant)'''
    e1, e2 = np.array([1, 0]), np.array([0, 1])
    A_eq = np.matrix(np.ones(HD_payoff_matrix.shape[1]))
    b_eq = np.matrix([[1]])
    A_ub = -(HD_payoff_matrix - threshold)
    b_ub = np.matrix(np.zeros(A_ub.shape[0])).T

    res_1 = linprog(c = e2, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq)
    res_2 = linprog(c = e1, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq)
    if not res_1.success: print("Note that the LP for e1 is not successful...")
    if not res_2.success: print("Note that the LP for e2 is not successful...")
    limiting_strategy_1 = np.matrix(res_1.x).T
    limiting_strategy_2 = np.matrix(res_2.x).T

    return [limiting_strategy_1, limiting_strategy_2]

s1, s2 = safety_zone(11)
print("If the state falls in the convex hull of these two, then the population will not extinct:")
print(s1.T, s2.T, sep = "\n", end = "\n\n")

print("The fitness for the two birds repectively are:")
print(A*s1, A*s2, sep = "\n\n")

If the state falls in the convex hull of these two, then the population will not extinct:
[[0.4 0.6]]
[[2.39306219e-09 9.99999998e-01]]

The fitness for the two birds repectively are:
[[17.]
 [11.]]

[[44.99999984]
 [14.99999998]]


In [11]:
population = Population(initial_hawk_frequency = rd.random(), size = 256)
hawk_frequency_history = {}
for t in range(8):
    hawk_frequency_history[t] = population.frequency("Hawk")
    exists = population.evolve(threshold = 20, 
                        loop = 4, reset_before_each_loop = True, method = "round_robin")
    if not exists: break
hawk_frequency_history = pd.Series(hawk_frequency_history)
print(hawk_frequency_history)

0    0.085938
dtype: float64


In [12]:
threshold_axis = np.linspace(-40, 20, 64)

last_time = time.time()

for i, threshold_i in enumerate(threshold_axis):
    s1, s2 = safety_zone(threshold_i)
    loop_index = str(i)
    if len(loop_index) < 2: loop_index = '0' + loop_index
    result_file = open("./Data/Result_{}_{}.csv".format(loop_index, threshold_i), 'w')
    result_file.write("{}\n".format(s1.T))
    result_file.write("{}\n\n".format(s2.T))
    result_file.write("Initial_Hawk_Frequency,Final_Hawk_Frequency,Evolution_Rounds,Hawk_Extincts,Dove_Extincts,\n")
    for j in range(128):
        initial_hawk_frequency = rd.random()
        population = Population(initial_hawk_frequency = initial_hawk_frequency, size = 128)
        hawk_frequency_history = {}
        for t in range(16):
            hawk_frequency_history[t] = population.frequency("Hawk")
            exists = population.evolve(threshold = threshold_i, 
                                loop = 1, reset_before_each_loop = True, method = "round_robin")
            if not exists: break
        hawk_frequency_history = pd.Series(hawk_frequency_history)
        final_hawk_frequency = hawk_frequency_history.tail(1).iloc[0]
        evolution_rounds = hawk_frequency_history.shape[0]
        hawk_extincts = int(final_hawk_frequency == 0)
        dove_extincts = int(final_hawk_frequency == 1)
        result_file.write("{},{},{},{},{},\n".format(initial_hawk_frequency,
                                                     final_hawk_frequency,
                                                     evolution_rounds,
                                                     hawk_extincts,
                                                     dove_extincts))
    result_file.close()
    print("Threshold {} took: {}s".format(i, time.time() - last_time))
    last_time = time.time()

Threshold 0 took: 17.48354721069336s
Threshold 1 took: 16.9706974029541s
Threshold 2 took: 17.1657075881958s
Threshold 3 took: 16.106345176696777s
Threshold 4 took: 16.610971689224243s
Threshold 5 took: 16.76271653175354s
Threshold 6 took: 16.897802591323853s
Threshold 7 took: 16.800772428512573s
Threshold 8 took: 16.59148406982422s
Threshold 9 took: 16.568203449249268s
Threshold 10 took: 17.264357566833496s
Threshold 11 took: 16.90040636062622s
Threshold 12 took: 17.549776077270508s
Threshold 13 took: 16.831238746643066s
Threshold 14 took: 17.44094944000244s
Threshold 15 took: 17.034124851226807s
Threshold 16 took: 16.396556615829468s
Threshold 17 took: 17.655654668807983s
Threshold 18 took: 17.68483853340149s
Threshold 19 took: 17.303399085998535s
Threshold 20 took: 17.31617283821106s
Threshold 21 took: 17.40225076675415s
Threshold 22 took: 17.15489673614502s
Threshold 23 took: 17.29096031188965s
Threshold 24 took: 18.18210244178772s
Threshold 25 took: 17.537060022354126s
Threshold 2